Выбор модели для предсказания модели. 

Грузим данные.

In [1]:
import pickle
with open("data_5.pkl", 'rb') as f:
    X_pairs, y = pickle.load(f)

In [2]:
len(X_pairs), len(y)

(105023, 105023)

0-й и 8-й элементы - это текстовые данные. 

In [3]:
x = X_pairs[0]

x[0][0], x[1][0], x[0][8],x[1][8]

('просторный таунхаус  с отдельно стоящим гаражом на 2 машины площадь около 30  квм в посёлке ангеловорезиденц на пятницком шоссе основа дома  монолитный каркас с отделкой кирпичом в доме 3 этажа включая мансарду и подвал который по площади как первый этаж всего на этих трёх этажах 8 комнат на первом этаже прихожая выход на террасу кухня полностью оборудованная гостиная и зал с камином кабинет гостевой санузелна втором этаже родительская спальня с гардеробной комнатой и санузлом с душевой кабиной и ванной 2 спальни со смежным санузлом с душевой кабиной в каждой спальне встроенные шкафы для вещей кабинетспальня со встроенным шкафом санузел с душевой кабинойна третьем этаже спальня с кладовкой и санузлом с душевой кабиной комната свободного назначения с санузломна цокольном этажев подвале прачечная со стиральной и сушильной машинами 7 комнат свободного назначения сейчас используются для хранения личных вещей бойлернаятехническое помещение с отдельной дверью для доступа техническихгазовых

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train_pairs, X_test_pairs, y_train, y_test = train_test_split(X_pairs, y)

Генерируем текстовый корпус из всех текстовых данных тренировочного сета.

In [7]:
text_data = []

for x in X_train_pairs:
    text_data.append(x[0][0])
    text_data.append(x[1][0])
    text_data.append(x[0][8])
    text_data.append(x[1][8])
    
text_data = list(set(text_data))
print(len(text_data))

28435


Обучим фаст текст на нашем корпусе.

In [8]:
from fasttext_classifier import FastTextClassifier

In [9]:
ft = FastTextClassifier(epoch=10, dim=100)
ft.fit(text_data)
ft.save("ft_model.bin")

Train unsupervised model.
Model trained with user parameters:
{'epoch': 10, 'dim': 100}.


Сохраняем модель.

In [9]:
ft = FastTextClassifier(epoch=10, dim=100)
ft.load("ft_model.bin")

Генерация данных.

Для текстовых данных мы считаем косинусное расстояние между энбеддингами предложений пары.

Для числовых данных считаем абсолютную разность.

Для всех других просто смотрим их равенство.

In [10]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))

Данные ледат в таком порядке:

1. description
2. roomscount
3. floornumber
4. category
5. totalarea
6. flattype
7. userid
8. publisheduseri
9. userInput
10. floorsCount
11. totalArea
12. price

In [11]:
import numpy as np

def create_data(X_pairs, ft):

    X = []

    for ix, x in enumerate(X_pairs):
        name_cos = cos_sim(ft.transform(x[0][0]), ft.transform(x[1][0]))
        addr_cos = cos_sim(ft.transform(x[0][8]), ft.transform(x[1][8]))
        same_roomscount  = int(x[0][1] == x[1][1])
        same_floornumber = int(x[0][2] == x[1][2])
        same_category    = int(x[0][3] == x[1][3])
        same_totalarea   = int(x[0][4] == x[1][4] )
        same_flattype    = int(x[0][5] == x[1][5] )
        same_userid      = int(x[0][6] == x[1][6] )
        same_publisheduseri = int(x[0][7] == x[1][7])
        diff_floorsCount    = np.abs(x[0][9] - x[1][9])
        diff_totalArea      = np.abs(x[0][10] - x[1][10])
        diff_price          = np.abs(x[0][11] - x[1][11])

        X.append([name_cos, addr_cos, same_roomscount, same_floornumber, same_category, same_totalarea, same_flattype, 
    same_userid, same_publisheduseri, diff_floorsCount, diff_totalArea, diff_price])
    return X

In [12]:
X_train = np.array(create_data(X_train_pairs, ft))
X_test = np.array(create_data(X_test_pairs, ft))

In [13]:
X_train[90]

array([9.00978088e-01, 8.39517057e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.20000000e+01, 1.00000000e+02, 1.37000000e+06])

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

### LogReg

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

ss = StandardScaler().fit(X_train)

X_train_s = ss.transform(X_train)
X_test_s = ss.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_s, y_train)
y_pred = clf.predict(X_test_s)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     23720
           1       0.95      0.95      0.95      2536

    accuracy                           0.99     26256
   macro avg       0.97      0.97      0.97     26256
weighted avg       0.99      0.99      0.99     26256

[[23591   129]
 [  135  2401]]


## Random forest

In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23720
           1       0.98      0.97      0.97      2536

    accuracy                           1.00     26256
   macro avg       0.99      0.99      0.99     26256
weighted avg       1.00      1.00      1.00     26256

[[23657    63]
 [   64  2472]]


Сохраняем модель RF так как она показала отличные результаты.

In [17]:
with open("model.pkl", "wb") as f:
    pickle.dump(clf, f)